In [7]:
import pandas as pd
import numpy as np
import torch as t
import plotly.graph_objects as go
import seaborn as sns

In [33]:
# load in clean dataset
clean_data = pd.read_csv(r"/home/tobi/Desktop/Capstone/EnergyDemandForecasting/src/Saved/Datasets/clean_training.csv", index_col=0)
clean_data.index = pd.to_datetime(clean_data.index)
clean_data.loc[:,"HourlyPrecipitation"] = clean_data["HourlyPrecipitation"].replace({np.nan:"None"})
display(clean_data)

,Energy Demand (MWH),HourlyDryBulbTemperature,HourlyDewPointTemperature,HourlyStationPressure,HourlyPrecipitation,HourlyWindSpeed,Energy Price (cents/KWH),Labor Force Level,Civilian Noninstitutional Population,Labor Force Participation
2018-06-19 05:00:00,7221.0,78.0,72.0,29.62,None,6.000000,19.28,4272100.0,7105823.0,60.1
2018-06-19 06:00:00,6911.0,79.0,70.0,29.63,None,6.000000,19.28,4272100.0,7105823.0,60.1
2018-06-19 07:00:00,6691.0,81.0,69.0,29.63,None,3.544254,19.28,4272100.0,7105823.0,60.1
2018-06-19 08:00:00,6582.0,82.0,67.0,29.63,None,3.789006,19.28,4272100.0,7105823.0,60.1
2018-06-19 09:00:00,6600.0,81.0,69.0,29.65,None,7.000000,19.28,4272100.0,7105823.0,60.1
...,...,...,...,...,...,...,...,...,...,...
2023-08-05 14:00:00,6168.0,84.0,58.0,29.81,None,6.000000,22.33,4165918.0,6747734.0,61.7
2023-08-05 15:00:00,6431.0,83.0,60.0,29.82,None,7.000000,22.33,4165918.0,6747734.0,61.7
2023-08-05 16:00:00,6599.0,82.0,55.0,29.82,None,0.000000,22.33,4165918.0,6747734.0,61.7
2023-08-05 17:00:00,6745.0,81.0,56.0,29.82,None,5.000000,22.33,4165918.0,6747734.0,61.7


In [118]:
input_data = clean_data.copy()
output_data = clean_data.copy()

In [119]:
# encode hour of the day, day of the week, and day of the year in terms of sine and cosine
input_data.loc[:,"Hour of Day"] = input_data.index.hour
input_data.loc[:,"Day of Week"] = input_data.index.dayofweek
input_data.loc[:,"Day of Year"] = input_data.index.dayofyear
display(input_data)

# should also try doing this without sine and cosine and compare the difference

,Energy Demand (MWH),HourlyDryBulbTemperature,HourlyDewPointTemperature,HourlyStationPressure,HourlyPrecipitation,HourlyWindSpeed,Energy Price (cents/KWH),Labor Force Level,Civilian Noninstitutional Population,Labor Force Participation,Hour of Day,Day of Week,Day of Year
2018-06-19 05:00:00,7221.0,78.0,72.0,29.62,None,6.000000,19.28,4272100.0,7105823.0,60.1,5,1,170
2018-06-19 06:00:00,6911.0,79.0,70.0,29.63,None,6.000000,19.28,4272100.0,7105823.0,60.1,6,1,170
2018-06-19 07:00:00,6691.0,81.0,69.0,29.63,None,3.544254,19.28,4272100.0,7105823.0,60.1,7,1,170
2018-06-19 08:00:00,6582.0,82.0,67.0,29.63,None,3.789006,19.28,4272100.0,7105823.0,60.1,8,1,170
2018-06-19 09:00:00,6600.0,81.0,69.0,29.65,None,7.000000,19.28,4272100.0,7105823.0,60.1,9,1,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-05 14:00:00,6168.0,84.0,58.0,29.81,None,6.000000,22.33,4165918.0,6747734.0,61.7,14,5,217
2023-08-05 15:00:00,6431.0,83.0,60.0,29.82,None,7.000000,22.33,4165918.0,6747734.0,61.7,15,5,217
2023-08-05 16:00:00,6599.0,82.0,55.0,29.82,None,0.000000,22.33,4165918.0,6747734.0,61.7,16,5,217
2023-08-05 17:00:00,6745.0,81.0,56.0,29.82,None,5.000000,22.33,4165918.0,6747734.0,61.7,17,5,217


In [120]:
if t.cuda.is_available():
    device = t.device("cuda")
    print("PyTorch is using:", t.cuda.get_device_name(0))
else:
    print("PyTorch is using CPU")

PyTorch is using: Radeon RX 7900 XTX


Encode Categorical Variables

In [121]:
input_data = pd.get_dummies(input_data, drop_first=True).astype("float32")
output_data = pd.get_dummies(output_data, drop_first=True).astype("float32")

Normalize features into [0,1]

In [122]:
input_min_vals = np.min(input_data, axis=0)
input_max_vals = np.max(input_data, axis=0)
output_min_vals = np.min(output_data, axis=0)
output_max_vals = np.max(output_data, axis=0)
# Normalize each column to be in the range [0, 1]
input_data = (input_data - input_min_vals) / (input_max_vals - input_min_vals)
output_data = (output_data - output_min_vals) / (output_max_vals - output_min_vals)

In [196]:
def format_data_lstm(input_data:pd.DataFrame, output_data:pd.DataFrame, sequence_length:int, batch_size:int):
    pass


# formatted dataset: (N x B x S x K)
B = 8   # Batch size
S = 168   # Sequence length
K = input_data.shape[1]  # Number of features

# Reshape data to have dimensions (N x B x S x K)
x = input_data.values
y = output_data.values
# reshaped_data = np.tile(data, (N,B,S,K)) #.reshape(N, B, S, K)

# Calculate the number of sequences of length S that can be produced
num_sequences = x.shape[0] - S

# Initialize an empty list to store the groups
x_inputs = []
y_outputs = []

# Iterate over the array to create groups
for i in range(num_sequences):
    input = x[i:i+S]
    output = y[i+S]
    x_inputs.append(input)
    y_outputs.append(output)

# Convert the list of groups to a NumPy array
x = np.array(x_inputs)
y = np.array(y_outputs)
x = x[(x.shape[0]%B):]
y = y[(y.shape[0]%B):]
x = np.array(np.split(x, x.shape[0]//B, axis=0))
y = np.array(np.split(y, y.shape[0]//B, axis=0))

# display(data.shape)
display(x.shape)
display(y.shape)
# print(formatted_data)

(5599, 8, 168, 15)

(5599, 8, 12)

Format using DataLoader

In [209]:
# formatted dataset: (N x B x S x K)
B = 8   # Batch size
S = 168   # Sequence length
K = input_data.shape[1]  # Number of features

# Reshape data to have dimensions (N x B x S x K)
x = input_data.values
y = output_data.values

# Calculate the number of sequences of length S that can be produced
num_sequences = x.shape[0] - S

# Initialize an empty list to store the groups
x_inputs = []
y_outputs = []

# Iterate over the array to create groups
for i in range(num_sequences):
    input = x[i:i+S]
    output = y[i+S]
    x_inputs.append(input)
    y_outputs.append(output)

x_inputs = t.Tensor(np.array(x_inputs))
y_outputs = t.Tensor(np.array(y_outputs))
train_dataset = t.utils.data.TensorDataset(x_inputs, y_outputs)
train_loader = t.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=False)

In [212]:
for input, output in train_loader:
    print(input.shape)
    print(output.shape)
    break

# HERE: validating format. Then need to format into 10% validation dataset and 90% training
display(train_loader.dataset[0])

# Test formatting
y_index = np.random.randint(y.shape[0]*y.shape[1]-1)
y_sample_index = y_index//B
y_batch_index = y_index%B

x_index = y_index + 1
x_sample_index = x_index//B
x_batch_index = x_index%B

display(x[x_sample_index,x_batch_index,-1] * (input_max_vals-input_min_vals) + input_min_vals)
display(y[y_sample_index,y_batch_index] * (output_max_vals-output_min_vals) + output_min_vals)

torch.Size([8, 168, 15])
torch.Size([8, 12])


(tensor([[0.4845, 0.8172, 0.9271,  ..., 0.0000, 0.0000, 1.0000],
         [0.4430, 0.8280, 0.9062,  ..., 0.0000, 0.0000, 1.0000],
         [0.4135, 0.8495, 0.8958,  ..., 0.0000, 0.0000, 1.0000],
         ...,
         [0.4397, 0.6774, 0.7083,  ..., 0.0000, 0.0000, 1.0000],
         [0.3924, 0.6559, 0.7188,  ..., 0.0000, 0.0000, 1.0000],
         [0.3271, 0.6344, 0.7396,  ..., 0.0000, 0.0000, 1.0000]]),
 tensor([0.2675, 0.6774, 0.7396, 0.7126, 0.3500, 0.3423, 0.9246, 0.9623, 0.8265,
         0.0000, 0.0000, 1.0000]))

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [197]:
display(clean_data.iloc[S+S:])

,Energy Demand (MWH),HourlyDryBulbTemperature,HourlyDewPointTemperature,HourlyStationPressure,HourlyPrecipitation,HourlyWindSpeed,Energy Price (cents/KWH),Labor Force Level,Civilian Noninstitutional Population,Labor Force Participation
2018-07-03 05:00:00,8149.0,80.0,74.0,30.01,None,0.0,19.37,4271649.0,7107053.0,60.1
2018-07-03 06:00:00,7774.0,82.0,75.0,30.01,None,0.0,19.37,4271649.0,7107053.0,60.1
2018-07-03 07:00:00,7527.0,84.0,75.0,30.01,None,0.0,19.37,4271649.0,7107053.0,60.1
2018-07-03 08:00:00,7374.0,86.0,76.0,30.01,None,0.0,19.37,4271649.0,7107053.0,60.1
2018-07-03 09:00:00,7355.0,89.0,76.0,30.03,None,3.0,19.37,4271649.0,7107053.0,60.1
...,...,...,...,...,...,...,...,...,...,...
2023-08-05 14:00:00,6168.0,84.0,58.0,29.81,None,6.0,22.33,4165918.0,6747734.0,61.7
2023-08-05 15:00:00,6431.0,83.0,60.0,29.82,None,7.0,22.33,4165918.0,6747734.0,61.7
2023-08-05 16:00:00,6599.0,82.0,55.0,29.82,None,0.0,22.33,4165918.0,6747734.0,61.7
2023-08-05 17:00:00,6745.0,81.0,56.0,29.82,None,5.0,22.33,4165918.0,6747734.0,61.7


In [198]:
# Test formatting
y_index = np.random.randint(y.shape[0]*y.shape[1]-1)
y_sample_index = y_index//B
y_batch_index = y_index%B

x_index = y_index + 1
x_sample_index = x_index//B
x_batch_index = x_index%B

display(x[x_sample_index,x_batch_index,-1] * (input_max_vals-input_min_vals) + input_min_vals)
display(y[y_sample_index,y_batch_index] * (output_max_vals-output_min_vals) + output_min_vals)

Energy Demand (MWH)                     7.606000e+03
HourlyDryBulbTemperature                8.200000e+01
HourlyDewPointTemperature               6.900000e+01
HourlyStationPressure                   2.979000e+01
HourlyWindSpeed                         5.000000e+00
Energy Price (cents/KWH)                1.937000e+01
Labor Force Level                       4.271649e+06
Civilian Noninstitutional Population    7.107053e+06
Labor Force Participation               6.010000e+01
Hour of Day                             1.600000e+01
Day of Week                             5.000000e+00
Day of Year                             2.090000e+02
HourlyPrecipitation_Light Rain          0.000000e+00
HourlyPrecipitation_Medium Rain         0.000000e+00
HourlyPrecipitation_None                1.000000e+00
dtype: float32

Energy Demand (MWH)                     7.606000e+03
HourlyDryBulbTemperature                8.200000e+01
HourlyDewPointTemperature               6.900000e+01
HourlyStationPressure                   2.979000e+01
HourlyWindSpeed                         5.000000e+00
Energy Price (cents/KWH)                1.937000e+01
Labor Force Level                       4.271649e+06
Civilian Noninstitutional Population    7.107053e+06
Labor Force Participation               6.010000e+01
HourlyPrecipitation_Light Rain          0.000000e+00
HourlyPrecipitation_Medium Rain         0.000000e+00
HourlyPrecipitation_None                1.000000e+00
dtype: float32

In [205]:
import numpy as np

class LSTM(t.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout:float = 0):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = t.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = t.nn.Dropout(dropout)
        self.fc = t.nn.Sequential(
            # t.nn.Linear(hidden_size, hidden_size),
            t.nn.Linear(hidden_size, output_size),
            t.nn.Softplus() 
        ) 
        self.c0 = None
        self.h0 = None

    def forward(self, x, save_cell_state:bool=False, bayesian_predict:bool=False):
        device = next(self.parameters()).device
        if save_cell_state: # Use previous hidden state and cell state
            if self.c0 is None: # no previous states so need to initialize them
                self.h0 = t.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
                self.c0 = t.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
            h0 = self.c0
            c0 = self.h0
        else: # Initialize hidden state and cell state
            h0 = t.zeros(self.num_layers, x.size(0), self.hidden_size, device=device)
            c0 = t.zeros(self.num_layers, x.size(0), self.hidden_size, device=device)
            # h0 = t.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
            # c0 = t.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)


        # Forward propagate LSTM
        out, (h0, c0) = self.lstm(x, (h0, c0))

        # Decode the hidden state of the last time step
        if self.eval or bayesian_predict:
            out = self.dropout(out)
        out = self.fc(out[:, -1, :])
        return out

# Example usage:
input_size = x.shape[3]  # Number of features in the input
hidden_size = 64  # Number of LSTM units (hidden size)
num_layers = 1  # Number of LSTM layers
output_size = y.shape[2]  # Number of output predictions
seq_length = S  # Length of input sequence (assuming hourly data)


# Forward pass
model = LSTM(input_size, hidden_size, num_layers, output_size)
input_tensor = t.tensor(x[0], device=next(model.parameters()).device) #.to(device="cuda")
output = model(input_tensor)
print("Output shape:", output.shape)
print(y[0].shape)


Output shape: torch.Size([8, 12])
(8, 12)


Train

In [203]:
display(x[0].shape)

(8, 168, 15)

In [206]:
# set device
device = "cuda"
# device = "cpu"
# Initialize the LSTM model
model = LSTM(input_size, hidden_size, num_layers, output_size).to(device=device)

# pull out last 10% of data to use for early stopping validation

# define optimizer
criterion = t.nn.MSELoss(reduction="none")
optimizer = t.optim.Adam(model.parameters(), lr=0.001, weight_decay=0)
x_device = t.Tensor(x).to(device=device)
y_device = t.Tensor(y).to(device=device)
weights = t.Tensor([1]+[0.1]*(y_device.shape[2]-1)).to(device=device)

for epoch in range(50):
    for b in range(x.shape[0]):
        outputs = model(x_device[b])
        # outputs = model(x_gpu[b])
        optimizer.zero_grad()
        loss = criterion(y_device[b], outputs) * 1000
        
        loss = loss * weights[None,:]
        loss = t.mean(loss)
        # loss = criterion(y_gpu[b], outputs)
        loss.backward()
        # Print gradients of model parameters
        if (epoch % 25 == 0) & (b == 0):
            for name, param in model.named_parameters():
                if param.grad is not None:
                    print(f"Gradient of {name}:")
                    print(param.grad)
                    print(param)
        optimizer.step()
        # print("Batch {}, Loss: {:.2f}".format(b+1, loss.item()))
    # print("NEW EPOCH\n\n\n")
    # if epoch == 1: break
    print("Epoch {}, Loss: {}".format(epoch+1, loss.item()))

Gradient of lstm.weight_ih_l0:
tensor([[-1.7076e-02, -3.8680e-02, -3.7660e-02,  ..., -1.1561e-23,
          5.2180e-17, -5.0460e-02],
        [ 2.2938e-02,  4.9985e-02,  4.8285e-02,  ..., -5.6351e-24,
         -3.4475e-17,  6.4374e-02],
        [-9.8307e-03, -2.3327e-02, -2.2388e-02,  ...,  8.0854e-24,
         -4.0428e-17, -2.9939e-02],
        ...,
        [-5.0871e-02, -1.2321e-01, -1.1871e-01,  ..., -1.2735e-23,
         -7.5581e-17, -1.5939e-01],
        [-1.0151e-02, -2.1815e-02, -2.0968e-02,  ...,  4.1746e-24,
         -2.7111e-17, -2.7976e-02],
        [ 1.1850e-02,  2.8599e-02,  2.7421e-02,  ...,  1.5523e-23,
         -4.7278e-17,  3.6792e-02]], device='cuda:0')
Parameter containing:
tensor([[-0.0986,  0.1234,  0.0072,  ...,  0.0860, -0.0230, -0.0665],
        [-0.0555,  0.0840,  0.0473,  ..., -0.0352, -0.0061, -0.0499],
        [ 0.0941,  0.1125,  0.1079,  ...,  0.0932, -0.0646, -0.0367],
        ...,
        [-0.1078,  0.0056,  0.0823,  ...,  0.1141,  0.0548, -0.0475],
     

KeyboardInterrupt: 

In [186]:
display(model(x_cpu[0])[:,0] * (input_max_vals[0]-input_min_vals[0]) + input_min_vals[0])
display(y_cpu[0,:,0] * (output_max_vals[0]-output_min_vals[0]) + output_min_vals[0])

/tmp/ipykernel_6902/3182508148.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  display(model(x_cpu[0])[:,0] * (input_max_vals[0]-input_min_vals[0]) + input_min_vals[0])


tensor([5440.6001, 6054.9644, 6467.3330, 6821.3594, 7018.7861, 7094.3936,
        7150.6680, 7156.3184, 7071.3867, 6943.8691, 6972.5518, 7253.1006,
        6737.5771, 6595.3301, 6470.1401, 6394.1025, 6351.9033, 6126.7974,
        5613.6479, 5199.7451, 5301.0952, 5218.6616, 5398.1396, 5377.5811,
        5692.0078, 6313.6396, 6709.7588, 6956.5205, 7422.0918, 7834.5322,
        8234.8008, 8217.6602], grad_fn=<AddBackward0>)

/tmp/ipykernel_6902/3182508148.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  display(y_cpu[0,:,0] * (output_max_vals[0]-output_min_vals[0]) + output_min_vals[0])


tensor([5620., 6201., 6677., 6988., 7144., 7222., 7274., 7251., 7142., 7094.,
        7105., 7021., 6800., 6681., 6583., 6493., 6218., 5889., 5567., 5330.,
        5191., 5122., 5139., 5395., 5930., 6488., 6885., 7313., 7715., 8010.,
        8294., 8490.])

Rolling Cross Validation